---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [92]:
import pandas as pd
import numpy as np
x=3
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc, name="foo")
df.head(5)

0    03/25/93 Total time of visit (in minutes):\n                    
1    6/18/85 Primary Care Doctor:\n                                  
2    sshe plans to move as of 7/8/71 In-Home Services: None\n        
3    7 on 9/27/75 Audit C Score Current:\n                           
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
Name: foo, dtype: object

In [93]:
def date_sorter(): 
    #cleanup typos
    df[370]= df[370].replace('4 (9/1975)','4/9/1975 ')
    
    #04/20/2009; 04/20/09; 4/20/09; 4/3/09
    ans = df.str.extractall(r'(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2,4})')
    ans = ans.query('match == 0')
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #Verified Mar 20, 2009;  Mar 20 2009; Mar. 20, 2009;March 20, 2009;
    ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z\s,.-]*(?P<Day>\d{1,2})[-\s,]*?\s(?P<Year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Day>\d{1,2})[\s](?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z,.\s]*(?P<Year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #Feb 2009; Sep 2009; Oct 2010
    ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z.,\s]*(?P<Year>(?:19|20)\d{2})'))
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #6/2008; 12/2009
    ans = ans.append(df[not_duplicated].str.extractall(r'(?:[\sA-Za-z-]|^|A-Z|a-z)(?P<Month>\d{1,2})/(?P<Year>\d{4})'))
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #2009; 2010
    ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Year>(?:19|20)\d{2})'))
    not_duplicated = ~df.index.isin([index[0] for index in ans.index])
    
    #replace NaN Day and Month with 1
    ans['Day'].fillna(1, inplace=True)
    ans['Month'].fillna(1, inplace=True)

    # Year - change 86 to 1986
    ans['Year'] = ans['Year'].apply(lambda x: '19' + x if len(x) == 2 else x)

    # Month change 05 to 5
    ans['Month'] = ans['Month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)

    month_dict = {'Jan': 1, 'January': 1,'Feb': 2,'February': 2, 'Mar': 3, 'March': 3, 'Apr': 4, 'April': 4, 'May': 5, 'May,': 5, 'Jun': 6, 'June': 6, 'Jul': 7, 'July': 7, 'Aug':8, 'August': 8, 'Sep': 9, 'September': 9, 'Oct': 10,'October': 10, 'Nov': 11, 'November': 11,'Dec': 12, 'December': 12}
        
    ans = ans.replace(month_dict)

    ans['Month'] = ans['Month'].apply(lambda x: str(x))
    ans['Day'] = ans['Day'].apply(lambda x: str(x))
    ans['Year'] = ans['Year'].apply(lambda x: str(x))
    ans['Date'] = ans['Month'] + '/' + ans['Day'] + '/' + ans['Year']
    ans['Date'] = pd.to_datetime(ans['Date'])
    ans = ans.sort_values(by='Date')
    rank = pd.Series(list(ans.index.labels[0]))
    return rank

In [94]:
#helper snippets
#pd.set_option('display.max_colwidth', -1)
#df.iloc[[25,39,40, 71, 72]]
#date_sorter()
df[370]= df[370].replace('4 (9/1975)','4/9/1975 ')
df[370]

"4/9/1975 Patient's thoughts about current substance abuse:\n"

In [95]:
#junk
#df.str.extractall(r'(?P<Date>\d{1,2}/\d{1,2}/\d{2,4})')
#df_ans = df['text'].str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})')
#df.str.extractall(r'(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4})')
#df.str.extractall(r'((?P<Day>\d{2,4})? (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{2,4} )?\d{4}?)')
#df.str.extractall(r'((?P<Day>\d{2,4})? (?P<Month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*  (?:\d{2,4})')
#df.str.extractall(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\s,.-]?(?P<Day>\d{1,2})[-\s,]*?\s(?P<Year>\d{4})')

In [96]:
#04/20/2009; 04/20/09; 4/20/09; 4/3/09
#df.str.extractall(r'(\d{1,2}/\d{2}/\d{4})')
ans = df.str.extractall(r'(?P<Month>\d{1,2})[/-](?P<Day>\d{1,2})[/-](?P<Year>\d{2,4})')
ans = ans.query('match == 0')
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
ans

,,Month,Day,Year
,match,,,
0,0,03,25,93
1,0,6,18,85
2,0,7,8,71
3,0,9,27,75
4,0,2,6,96
5,0,7,06,79
6,0,5,18,78
7,0,10,24,89
8,0,3,7,86


In [97]:
#Verified Mar 20, 2009;  Mar 20 2009; Mar. 20, 2009;March 20, 2009;
ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z\s,.-]*(?P<Day>\d{1,2})[-\s,]*?\s(?P<Year>\d{4})'))
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
np.sum(not_duplicated), len(not_duplicated)

(344, 500)

In [98]:
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
#ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Day>\d{1,2})[\s](?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]*)[\s](?P<Year>\d{4})'))
ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Day>\d{1,2})[\s](?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z,.\s]*(?P<Year>\d{4})'))
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
np.sum(not_duplicated), len(not_duplicated)

(275, 500)

In [99]:
#no instances exist: Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009; Mar-20-2009; 

In [100]:
#Feb 2009; Sep 2009; Oct 2010
ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z.,\s]*(?P<Year>(?:19|20)\d{2})'))
#ans = ans.append(df[not_duplicated].str.extractall(r'^(?:!\d\d\s)*(?P<Month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z,.]*)\s(?P<Year>(?:19|20)\d{2})'))
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
#6/2008; 12/2009
ans = ans.append(df[not_duplicated].str.extractall(r'(?:[\sA-Za-z-]|^|A-Z|a-z)(?P<Month>\d{1,2})/(?P<Year>\d{4})'))
check = ans
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
#2009; 2010
ans = ans.append(df[not_duplicated].str.extractall(r'(?P<Year>(?:19|20)\d{2})'))
not_duplicated = ~df.index.isin([index[0] for index in ans.index])
np.sum(not_duplicated), len(not_duplicated)
#[i for i, x in enumerate(not_duplicated) if x]
ans[300:400]

,,Day,Month,Year
,match,,,
303,0,NaN,Jan,1977
304,0,NaN,Mar,2002
305,0,NaN,Feb,2000
306,0,NaN,May,2004
307,0,NaN,Jul,2006
308,0,NaN,Feb,1994
309,0,NaN,Apr,1977
310,0,NaN,Oct,1992
311,0,NaN,Feb,1995


In [101]:
#replace NaN Day and Month with 1
ans['Day'].fillna(1, inplace=True)
ans['Month'].fillna(1, inplace=True)

# Year - change 86 to 1986
ans['Year'] = ans['Year'].apply(lambda x: '19' + x if len(x) == 2 else x)

# Month change 05 to 5
ans['Month'] = ans['Month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)

month_dict = {'Jan': 1, 'January': 1,'Feb': 2,'February': 2, 'Mar': 3, 'March': 3, 'Apr': 4, 'April': 4, 'May': 5, 'May,': 5, 'Jun': 6, 'June': 6, 'Jul': 7, 'July': 7, 'Aug':8, 'August': 8, 'Sep': 9, 'September': 9, 'Oct': 10,'October': 10, 'Nov': 11, 'November': 11,'Dec': 12, 'December': 12}
    
ans = ans.replace(month_dict)

ans['Month'] = ans['Month'].apply(lambda x: str(x))
ans['Day'] = ans['Day'].apply(lambda x: str(x))
ans['Year'] = ans['Year'].apply(lambda x: str(x))
ans['Month'] = ans['Month'].replace({',':''}, regex=True)
ans['Date'] = ans['Month'] + '/' + ans['Day'] + '/' + ans['Year']
ans['Date'] = pd.to_datetime(ans['Date'])
ans = ans.sort_values(by='Date')
rank = pd.Series(list(ans.index.labels[0]))
ans

,,Day,Month,Year,Date
,match,,,,
9,0,10,4,1971,1971-04-10
84,0,18,5,1971,1971-05-18
2,0,8,7,1971,1971-07-08
53,0,11,7,1971,1971-07-11
28,0,12,9,1971,1971-09-12
474,0,1,1,1972,1972-01-01
153,0,13,1,1972,1972-01-13
13,0,26,1,1972,1972-01-26
129,0,06,5,1972,1972-05-06


In [102]:
ans.to_csv ('ans.csv', header=True)
df.to_csv('df.csv',header=True)

In [103]:
ans.loc[313]

,Day,Month,Year,Date
match,,,,
0,1,12,1978,1978-12-01
